In [ ]:
import pandas as pd
import math

In [ ]:
retail_df = pd.read_excel('./12장_data/Online_Retail.xlsx')
retail_df.head()

In [ ]:
retail_df.info()

In [ ]:
#오류 데이터 정제(개수 및 가격이 음수 인 것, 회원ID가 없는 것)
#데이터 분석에 있어 결측치는 사용하지 않는다
retail_df = retail_df[retail_df['Quantity'] > 0]
retail_df = retail_df[retail_df['UnitPrice'] > 0]
retail_df = retail_df[retail_df['CustomerID'].notnull()]

#'CustomerID' 자료형을 정수형으로 변환
retail_df['CustomerID']=retail_df['CustomerID'].astype(int)

In [ ]:
retail_df.info()
print(retail_df.isnull().sum())
print(retail_df.shape)

In [ ]:
#중복 레코드 제거
retail_df.drop_duplicates(inplace=True)
print(retail_df.shape)

In [ ]:
pd.DataFrame([{'Product':len(retail_df['StockCode'].value_counts()), 'Transaction':len(retail_df['InvoiceNo'].value_counts()), 'Customer':len(retail_df['CustomerID'].value_counts())}], columns=['Product', 'Transaction', 'Customer'], index=['counts'])

In [ ]:
#주문 금액 컬럼 추가
retail_df['SaleAmount']=retail_df['UnitPrice']*retail_df['Quantity']
retail_df.head()

In [ ]:
aggregations = {
    'InvoiceNo':'count',
    'SaleAmount':'sum',
    'InvoiceDate':'max'
}
customer_df = retail_df.groupby('CustomerID').agg(aggregations)
customer_df = customer_df.reset_index()
customer_df.head()

In [ ]:
customer_df = customer_df.rename(columns = {'InvoiceNo':'Freq', 'InvoiceDate':'ElapseDays'})
customer_df.head()

In [ ]:
import datetime
customer_df['ElapseDays'] = datetime.datetime(2011,12,10) - customer_df['ElapseDays']
customer_df.head()

In [ ]:
customer_df['ElapseDays'] = customer_df['ElapseDays'].apply(lambda x: x.days+1)
customer_df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots()
ax.boxplot([customer_df['Freq'], customer_df['SaleAmount'],customer_df['ElapseDays']], sym='bo')
plt.xticks([1,2,3], ['Freq', 'SaleAmount', 'ElapseDays'])
plt.show()
#파란색 박스가 안 보일 정도로 찌부되고 점들이 많이 분포된 것은 그만큼 예외값(고르지 않은)이 많다는 것

In [ ]:
import numpy as np

customer_df['Freq_log'] = np.log1p(customer_df['Freq'])
customer_df['SaleAmount_log'] = np.log1p(customer_df['SaleAmount'])
customer_df['ElapseDays_log'] = np.log1p(customer_df['ElapseDays'])
customer_df.head()

In [ ]:
fig, ax = plt.subplots()
ax.boxplot([customer_df['Freq_log'], customer_df['SaleAmount_log'], customer_df['ElapseDays_log']], sym='bo')
plt.xticks([1,2,3], ['Freq_log', 'SaleAmount_log', 'ElapseDays_log'])
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

In [ ]:
X_features = customer_df[['Freq_log', 'SaleAmount_log', 'ElapseDays_log']].values

In [ ]:
from sklearn.preprocessing import StandardScaler
X_features_scaled = StandardScaler().fit_transform(X_features)

In [ ]:
distortions = []

for i in range(1, 11):
    kmeans_i = KMeans(n_clusters = i, random_state = 0)#모델 생성
    kmeans_i.fit(X_features_scaled)
    distortions.append(kmeans_i.inertia_)
    
plt.plot(range(1,11), distortions, marker = 'o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()
#엘보우(팔꿈치) 기법: 가장 크게 꺾이는 부분을 찾는 것(여기서는 3~5 즈음)

In [ ]:
kmeans = KMeans(n_clusters = 3, random_state = 0) #모델 생성(클러스터 3으로)
#모델 학습과 결과 예측(클러스터 레이블 생성)
Y_labels = kmeans.fit_predict(X_features_scaled)

In [ ]:
customer_df['ClusterLabel'] = Y_labels
customer_df.head()

In [ ]:
from matplotlib import cm

def silhouetteViz(n_cluster, X_features): 
    
    kmeans = KMeans(n_clusters=n_cluster, random_state=0)
    Y_labels = kmeans.fit_predict(X_features)
    
    silhouette_values = silhouette_samples(X_features, Y_labels, metric='euclidean')

    y_ax_lower, y_ax_upper = 0, 0
    y_ticks = []

    for c in range(n_cluster):
        c_silhouettes = silhouette_values[Y_labels == c]
        c_silhouettes.sort()
        y_ax_upper += len(c_silhouettes)
        color = cm.jet(float(c) / n_cluster)
        plt.barh(range(y_ax_lower, y_ax_upper), c_silhouettes,
                 height=1.0, edgecolor='none', color=color)
        y_ticks.append((y_ax_lower + y_ax_upper) / 2.)
        y_ax_lower += len(c_silhouettes)
    
    silhouette_avg = np.mean(silhouette_values)
    plt.axvline(silhouette_avg, color='red', linestyle='--')
    plt.title('Number of Cluster : '+ str(n_cluster)+'\n' \
              + 'Silhouette Score : '+ str(round(silhouette_avg,3)))
    plt.yticks(y_ticks, range(n_cluster))   
    plt.xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.ylabel('Cluster')
    plt.xlabel('Silhouette coefficient')
    plt.tight_layout()
    plt.show()

In [ ]:
def clusterScatter(n_cluster, X_features): 
    c_colors = []
    kmeans = KMeans(n_clusters=n_cluster, random_state=0)
    Y_labels = kmeans.fit_predict(X_features)

    for i in range(n_cluster):
        c_color = cm.jet(float(i) / n_cluster) #클러스터의 색상 설정
        c_colors.append(c_color)
        #클러스터의 데이터 분포를 동그라미로 시각화
        plt.scatter(X_features[Y_labels == i,0], X_features[Y_labels == i,1],
                     marker='o', color=c_color, edgecolor='black', s=50, 
                     label='cluster '+ str(i))       
    
    #각 클러스터의 중심점을 삼각형으로 표시
    for i in range(n_cluster):
        plt.scatter(kmeans.cluster_centers_[i,0], kmeans.cluster_centers_[i,1], 
                    marker='^', color=c_colors[i], edgecolor='w', s=200)
        
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
silhouetteViz(3, X_features_scaled) #클러스터 3개인 경우의 실루엣 score 및 각 클러스터 비중 시각화

In [ ]:
silhouetteViz(4, X_features_scaled) #클러스터 4개인 경우의 실루엣 score 및 각 클러스터 비중 시각화

In [ ]:
silhouetteViz(5, X_features_scaled) #클러스터 5개인 경우의 실루엣 score 및 각 클러스터 비중 시각화

In [ ]:
silhouetteViz(6, X_features_scaled) #클러스터 6개인 경우의 실루엣 score 및 각 클러스터 비중 시각화

In [ ]:
clusterScatter(3, X_features_scaled)

In [ ]:
clusterScatter(4, X_features_scaled)

In [ ]:
clusterScatter(5, X_features_scaled)

In [ ]:
clusterScatter(5, X_features_scaled)

In [ ]:
clusterScatter(6, X_features_scaled)

In [ ]:
best_cluster = 4

kmeans = KMeans(n_clusters=best_cluster, random_state=0)
Y_labels = kmeans.fit_predict(X_features_scaled)

In [ ]:
customer_df['ClusterLabel'] = Y_labels

customer_df.head()   #작업 확인용 출력

In [ ]:
customer_df.to_csv('./12장_data/Online_Retail_Customer_Cluster.csv')

In [ ]:
customer_df.groupby('ClusterLabel')['CustomerID'].count()

In [ ]:
customer_cluster_df = customer_df.drop(['Freq_log', 'SaleAmount_log', 'ElapseDays_log'],axis=1, inplace=False)

In [ ]:
# 주문 1회당 평균 구매금액 : SaleAmountAvg
customer_cluster_df['SaleAmountAvg'] = customer_cluster_df['SaleAmount']/customer_cluster_df['Freq']

customer_cluster_df.head()

In [ ]:
# 클러스터별 분석
customer_cluster_df.drop(['CustomerID'],axis=1, inplace=False).groupby('ClusterLabel').mean()